# 📘 Notebook 2: Boundary Setup

---

## 🎯 Purpose

This notebook handles **Phase 2: Boundary Management** - creating the geographic and administrative hierarchy for your tenant.

### What are Boundaries?

Boundaries represent the administrative divisions of your city/state:

```
City (Level 1)
  ├── Zone (Level 2)
      ├── Ward (Level 3)
          ├── Block (Level 4)
              └── Locality (Level 5)
```

**You can define your own hierarchy levels based on your needs.**

---

## ✅ Prerequisites

Before starting this notebook:

- [ ] **Completed Notebook 1** - Tenant must exist in system
- [ ] **Know your tenant code** (e.g., `pg.citya`, `statea`)
- [ ] **Boundary data ready** - Have your administrative divisions mapped
- [ ] **Authentication token** - Will need to re-authenticate

---

## 📋 Workflow Overview

### Step 1: Authenticate
Login to DIGIT Gateway (same as Notebook 1)

### Step 2: Select/Create Hierarchy
- **Option A:** Use existing hierarchy (if already defined)
- **Option B:** Create new hierarchy definition

### Step 3: Generate Template
System creates dynamic Excel template based on your hierarchy

### Step 4: Fill Boundary Data
Download and fill the generated Excel template

### Step 5: Upload & Process
Upload filled template - system validates and creates boundaries

### Step 6: Verify
Check boundary creation status and results

---

## 🗂️ Boundary Template Structure

The generated template will have columns like:

| Column | Description | Mandatory | Example |
|--------|-------------|-----------|----------|
| Country | Country name | Yes | India |
| State | State name | Yes | Punjab |
| City | City name | Yes | City A |
| Zone | Zone name | No | Zone 1 |
| Ward | Ward name/number | No | Ward 12 |
| Block | Sub-ward division | No | Block A |
| Code | Unique boundary code | Yes | PG_CITYA_ZONE1 |
| Parent Code | Parent boundary code | Required (except Level 1) | PG_CITYA |

**Note:** Columns vary based on your hierarchy definition.

---

## ⚠️ Important Rules

### Hierarchy Rules:
1. **Level 1 (City)** has NO parent
2. **All other levels** MUST have a valid parent code
3. **Parent must exist** before creating child
4. **Codes must be unique** across the tenant

### Example Valid Structure:
```
Code              | Name    | Parent Code
------------------|---------|-------------
PG_CITYA          | City A  | (empty)
PG_CITYA_Z1       | Zone 1  | PG_CITYA
PG_CITYA_Z1_W1    | Ward 1  | PG_CITYA_Z1
PG_CITYA_Z1_W1_B1 | Block 1 | PG_CITYA_Z1_W1
```

---

## 🔄 Processing & Validation

### Layer 1: Excel Validation
- Mandatory field checks
- Dropdown enforcement
- Regex validation for codes

### Layer 2: Schema Validation
- Correct column names
- Valid hierarchy levels
- Parent-child mapping rules
- Data type validation

### Layer 3: API Validation
- Parent code existence in DB
- Duplicate boundary checks
- Hierarchy type validation
- Async processing via Kafka

---

## 🚨 Common Errors

| Error Message | Cause | Solution |
|---------------|-------|----------|
| "Parent code not found" | Invalid parent reference | Ensure parent exists first |
| "Duplicate boundary code" | Code already used | Use unique codes |
| "Invalid hierarchy level" | Wrong level assignment | Follow hierarchy definition |
| "Circular reference" | Child references itself | Fix parent-child relationships |

---

## 📊 Status Tracking

After upload, you'll see:

- **Processing Started** - Upload accepted
- **Processing In Progress** - System creating boundaries
- **Processing Completed** - All boundaries created

If errors occur, download the error file showing:
- `_STATUS`: SUCCESS / FAILED
- `_STATUS_CODE`: HTTP code (201/400/409)
- `_ERROR_MESSAGE`: Detailed error description

---

**Ready to create boundaries? Let's begin! 👇**

In [ ]:
# ============================================================================
# 📦 AUTOMATIC DEPENDENCY INSTALLATION
# ============================================================================
# This cell automatically installs all required packages for cross-platform compatibility
# Supports: Windows, macOS (Intel & Apple Silicon), Linux (x86 & ARM)

import sys
import subprocess
import platform
import os

def get_platform_name():
    """Get user-friendly platform name"""
    system = platform.system()
    machine = platform.machine().lower()
    
    if system == "Darwin":
        if machine in ['arm64', 'aarch64']:
            return "macOS (Apple Silicon)"
        else:
            return "macOS (Intel)"
    elif system == "Windows":
        return "Windows"
    elif system == "Linux":
        if machine in ['arm64', 'aarch64']:
            return "Linux (ARM)"
        else:
            return "Linux"
    else:
        return f"{system} ({machine})"

def install_dependencies():
    """Install all required dependencies with cross-platform support"""
    
    print("=" * 70)
    print("🔧 CRS Data Loader - Dependency Installation")
    print("=" * 70)
    print(f"📍 Platform: {get_platform_name()}")
    print(f"🐍 Python: {sys.version.split()[0]}")
    print("=" * 70)
    
    # Check if requirements.txt exists (check parent dirs)
    req_file = 'requirements.txt'
    if not os.path.exists(req_file):
        req_file = '../requirements.txt'
    
    if os.path.exists(req_file):
        print(f"✅ Found requirements.txt - Installing from file...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", req_file, "--upgrade"])
            print("\n✅ Dependencies installed from requirements.txt")
        except subprocess.CalledProcessError:
            print(f"\n⚠️  Some packages failed, trying fallback...")
            install_fallback()
    else:
        print("⚠️  requirements.txt not found - Using fallback installation...")
        install_fallback()
    
    # Enable Jupyter widgets with enhanced Linux support
    print("\n🔧 Enabling Jupyter widgets...")
    widget_success = False
    
    # Step 1: Install widget packages
    try:
        print("   Installing widget packages...")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "ipywidgets", "jupyterlab_widgets", "--quiet"],
            timeout=30
        )
        widget_success = True
    except Exception:
        pass
    
    # Step 2: Try JupyterLab extension
    try:
        print("   Building JupyterLab extensions...")
        subprocess.check_call(
            [sys.executable, "-m", "jupyter", "lab", "build", "--minimize=False"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            timeout=90
        )
        print("   ✅ JupyterLab widgets installed")
        widget_success = True
    except Exception:
        print("   ⚠️  JupyterLab build skipped (may not be needed)")
    
    # Step 3: Try classic notebook extension
    try:
        subprocess.check_call(
            [sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            timeout=30
        )
        print("   ✅ Classic notebook widgets enabled")
        widget_success = True
    except Exception:
        pass
    
    if widget_success:
        print("\n✅ Jupyter widgets configured")
    
    print("\n" + "=" * 70)
    print("✅ INSTALLATION COMPLETE!")
    print("=" * 70)
    
    # Show restart message if widgets were configured
    if widget_success and platform.system() == "Linux":
        print("💡 On Linux: If widgets don't appear, please:")
        print("   1. Stop this notebook (Ctrl+C in terminal)")
        print("   2. Restart: jupyter lab (or jupyter notebook)")
        print("   3. Refresh your browser (Ctrl+Shift+R)")
        print("=" * 70)
    
    print("📝 You can now run the next cells")
    print("=" * 70)

def install_fallback():
    """Fallback installation if requirements.txt is missing"""
    
    core_packages = [
        "pandas>=2.0.0",
        "openpyxl>=3.1.0",
        "xlsxwriter>=3.1.0",
        "requests>=2.31.0",
        "python-dotenv>=1.0.0",
        "notebook>=7.0.0",
        "jupyterlab>=4.0.0",
        "ipykernel>=6.25.0",
        "ipywidgets>=8.1.0",
        "jupyterlab_widgets>=3.0.0",
        "tqdm>=4.65.0",
        "plotly>=5.17.0",
        "sqlalchemy>=2.0.0",
        "typing-extensions>=4.7.0"
    ]
    
    print("\n📦 Installing core packages...")
    for package in core_packages:
        try:
            pkg_name = package.split('>=')[0]
            print(f"   Installing {pkg_name}...", end=" ")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", package, "--quiet"],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print("✅")
        except subprocess.CalledProcessError:
            print("⚠️")
    
    print("\n📦 Installing database adapter...")
    machine = platform.machine().lower()
    if machine not in ['aarch64', 'arm64']:
        try:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "psycopg2-binary>=2.9.0", "--quiet"],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print("   ✅ psycopg2-binary installed")
        except:
            print("   ⚠️  psycopg2-binary skipped (optional)")

# Run installation
try:
    install_dependencies()
except Exception as e:
    print(f"\n❌ Installation failed: {str(e)}")
    print("\n💡 Manual installation - Run in terminal:")
    print("   pip install -r requirements.txt")
    print("   jupyter lab build")


In [ ]:
# Import required packages
import pandas as pd
import json
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
import shutil
import os
import sys
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Add parent directory to path to import unified_loader
# When running from Notebooks/ folder, parent is crs_dataLoader/
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Force reload modules to get latest changes
if 'unified_loader' in sys.modules:
    del sys.modules['unified_loader']
if 'mdms_validator' in sys.modules:
    del sys.modules['mdms_validator']
if 'excel_validator' in sys.modules:
    del sys.modules['excel_validator']

# Now import after adding to path
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans

# Load environment variables from parent directory
env_path = os.path.join(parent_dir, '.env')
load_dotenv(env_path)

# Initialize Global Variables
CONFIG = {}
CONFIG_SET = False
TENANT_FILE = None
COMMON_MASTER_FILE = None
UPLOADED_TENANTS = []
SELECTED_TENANT = None
UPLOADER = None  # Global authenticated uploader instance

# Clear upload folder (in parent directory)
upload_dir = os.path.join(parent_dir, 'upload')
if os.path.exists(upload_dir):
    for filename in os.listdir(upload_dir):
        file_path = os.path.join(upload_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    print(f"✅ Cleared all files from upload/")
else:
    os.makedirs(upload_dir)
    print(f"✅ Created upload/ directory")

print("✅ Packages loaded successfully!")
print("✅ Modules reloaded with latest changes!")
print(f"✅ Working directory: {os.getcwd()}")
print(f"✅ Parent directory (for imports): {parent_dir}")
print()
print("="*70)
print("  ⚙️ AUTHENTICATION REQUIRED")
print("="*70)
print("Please proceed to the next cell to authenticate with the gateway.")

In [ ]:
print("="*70)
print("        🔐  AUTHENTICATION")
print("="*70)
print()

# Authentication widgets
base_url_input = widgets.Text(
    value="",
    description='🌐 Domain URL:',
    placeholder='e.g., https://unified-dev.digit.org',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

username_input = widgets.Text(
    value="",
    description='👤 Username:',
    placeholder='e.g., DEV_SUPER_ADMIN',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

password_input = widgets.Password(
    value="",
    description='🔑 Password:',
    placeholder='Enter password',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

user_type_input = widgets.Dropdown(
    options=['EMPLOYEE', 'CITIZEN'],
    value="EMPLOYEE",
    description='👔 User Type:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tenant_id_input = widgets.Text(
    value="",
    description='🏛️ Tenant ID:',
    placeholder='e.g., dev, pg, pb',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

auth_button = widgets.Button(
    description='🔐 Authenticate',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

auth_output = widgets.Output()

def on_authenticate(b):
    global UPLOADER, CONFIG
    
    with auth_output:
        clear_output()
        
        # Validate inputs
        if not base_url_input.value.strip():
            print("❌ Gateway URL is required")
            return
        
        if not username_input.value.strip():
            print("❌ Username is required")
            return
        
        if not password_input.value.strip():
            print("❌ Password is required")
            return
        
        if not tenant_id_input.value.strip():
            print("❌ Tenant ID is required")
            return
        
        print("🔄 Authenticating with gateway...")
        print(f"   Gateway: {base_url_input.value}")
        print(f"   Username: {username_input.value}")
        print(f"   User Type: {user_type_input.value}")
        print(f"   Tenant ID: {tenant_id_input.value}")
        print()
        
        try:
            # Create APIUploader with authentication
            UPLOADER = APIUploader(
                base_url=base_url_input.value.strip(),
                username=username_input.value.strip(),
                password=password_input.value.strip(),
                user_type=user_type_input.value,
                tenant_id=tenant_id_input.value.strip()
            )
            
            if UPLOADER.authenticated:
                # Save config
                CONFIG = {
                    'base_url': base_url_input.value.strip(),
                    'tenant_id': tenant_id_input.value.strip()
                }
                CONFIG_SET = True
                
                print()
                print("="*70)
                print("  ✅ AUTHENTICATION SUCCESSFUL!")
                print("="*70)
                print(f"\n👤 Logged in as: {UPLOADER.user_info.get('userName', 'Unknown')}")
                print(f"📧 Email: {UPLOADER.user_info.get('emailId', 'N/A')}")
                print(f"🏛️ Tenant: {UPLOADER.user_info.get('tenantId', 'N/A')}")
                print(f"\n🎭 Roles:")
                for role in UPLOADER.user_info.get('roles', []):
                    print(f"   • {role.get('name', 'Unknown')} ({role.get('code', 'N/A')})")
                print()
                print("="*70)
                print("  ➡️ Proceed to PHASE 1: Tenant Setup")
                print("="*70)
            else:
                print("❌ Authentication failed. Please check your credentials.")
        
        except Exception as e:
            print(f"❌ Error during authentication: {str(e)}")

auth_button.on_click(on_authenticate)

# Display UI
auth_ui = widgets.VBox([
    widgets.HTML("<h3>🔐 Step 0:  Authentication</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Authenticate with the eGov gateway to access all services</i></p>"),
    widgets.HTML("<br>"),
    base_url_input,
    widgets.HTML("<p style='font-size: 11px; color: #888;'><i>💡 Domain URL - same for all services</i></p>"),
    widgets.HTML("<br>"),
    username_input,
    password_input,
    user_type_input,
    tenant_id_input,
    widgets.HTML("<p style='font-size: 11px; color: #ff6600;'><i>⚠️ Credentials are used only for this session and not stored</i></p>"),
    widgets.HTML("<br>"),
    auth_button,
    auth_output
])

display(auth_ui)

In [ ]:
# Dependency Check: Verify tenant exists
print("="*70)
print("        ✅ CHECKING PREREQUISITES")
print("="*70)
print()

# Check if authentication was successful
if not UPLOADER or not UPLOADER.authenticated:
    print("❌ ERROR: Authentication failed!")
    print("💡 Please check your credentials and try again.")
    raise SystemExit("Authentication required")

print("✅ Authentication successful")
print("\n📋 Checking tenant configuration...")

# Example tenant check (customize as needed)
# tenants = UPLOADER.search_mdms_data(schema_code='tenant.tenants', tenant='pg')
# if tenants:
#     print(f"✅ Found {len(tenants)} tenant(s) in system")
# else:
#     print("⚠️  Warning: No tenants found. You may need to run Notebook 1 first.")

print("\n✅ All prerequisites met - ready to proceed!")


# Phase 2: Boundary Master Data

---

# 🟩 PHASE 2: BOUNDARY MANAGEMENT WORKFLOW

This phase implements the complete boundary management workflow with template generation.

## 📋 Workflow Steps:

### **Option 1: Create New Hierarchy (First Time)**
1. Enter tenant ID and hierarchy type
2. Define boundary levels (City → Zone → Block → Locality)
3. Create the hierarchy definition
4. Generate and download Excel template
5. Fill template with boundary data
6. Upload and process filled template

### **Option 2: Use Existing Hierarchy**
1. Search for existing hierarchies
2. Select hierarchy and generate template
3. Download, fill, upload template

## 🎯 What You Need:
- **Tenant ID** (e.g., `pg`, `mz`, `pb`)
- **Hierarchy Type** (e.g., `ADMIN`, `ADMIN1`, `REVENUE`)
- **Boundary Levels** (e.g., City, Zone, Block, Locality)
- **Boundary data** to fill in the downloaded template

---

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from unified_loader import APIUploader
import time
import os

print("="*70)
print("   🔍 PHASE 2.1: BOUNDARY HIERARCHY & TEMPLATE GENERATION")
print("="*70)

# Global variables
BOUNDARY_TENANT = None
BOUNDARY_HIERARCHY_TYPE = None
TEMPLATE_FILESTORE_ID = None
DOWNLOADED_TEMPLATE_PATH = None
DOWNLOAD_URL = None


if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

uploader = UPLOADER

# UI Widgets
tenant_input = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    description='🏛️ Tenant ID:',
    placeholder='e.g., pg, mz, pb',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tab_selector = widgets.Tab()

# TAB 1: CREATE NEW HIERARCHY
hierarchy_type_input = widgets.Text(
    value='ADMIN',
    description='Hierarchy Type:',
    placeholder='e.g., ADMIN, ADMIN1',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

level_widgets = []
levels_container = widgets.VBox([])

add_level_btn = widgets.Button(
    description='➕ Add Level',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

create_hierarchy_btn = widgets.Button(
    description='🏗️ Create Hierarchy',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

generate_template_btn = widgets.Button(
    description='📄 Generate Template',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

download_template_btn = widgets.Button(
    description='📥 Download Template',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

create_output = widgets.Output()

# TAB 2: SEARCH EXISTING
search_btn = widgets.Button(
    description='🔍 Search Hierarchies',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

hierarchy_dropdown = widgets.Dropdown(
    options=[],
    description='Hierarchy:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%', visibility='hidden')
)

search_generate_btn = widgets.Button(
    description='📄 Generate Template',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

search_download_btn = widgets.Button(
    description='📥 Download Template',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

search_output = widgets.Output()

# Helper functions
def add_initial_level():
    global level_widgets
    level_widget = widgets.Text(
        value='',
        description='Level 1:',
        placeholder='e.g., City',
        style={'description_width': '140px'},
        layout=widgets.Layout(width='95%')
    )
    level_widgets.append(level_widget)
    levels_container.children = tuple(level_widgets)

def on_add_level_clicked(b):
    global level_widgets
    level_num = len(level_widgets) + 1
    level_widget = widgets.Text(
        value='',
        description=f'Level {level_num}:',
        placeholder=f'e.g., Zone, Block',
        style={'description_width': '140px'},
        layout=widgets.Layout(width='95%')
    )
    level_widgets.append(level_widget)
    levels_container.children = tuple(level_widgets)

add_level_btn.on_click(on_add_level_clicked)

def on_create_hierarchy_clicked(b):
    global BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE

    with create_output:
        clear_output()

        tenant = tenant_input.value.strip()
        if not tenant:
            print("❌ Please enter a tenant ID")
            return

        hierarchy_type = hierarchy_type_input.value.strip()
        if not hierarchy_type:
            print("❌ Please enter a hierarchy type")
            return

        levels = [lw.value.strip() for lw in level_widgets if lw.value.strip()]
        if len(levels) == 0:
            print("❌ Please add at least one boundary level")
            return

        BOUNDARY_TENANT = tenant
        BOUNDARY_HIERARCHY_TYPE = hierarchy_type

        print(f"🏗️ Creating boundary hierarchy...")
        print(f"   Tenant: {tenant}")
        print(f"   Hierarchy Type: {hierarchy_type}")
        print(f"   Levels: {' → '.join(levels)}")

        boundary_items = []
        prev_level = None

        for level in levels:
            boundary_items.append({
                "boundaryType": level,
                "parentBoundaryType": prev_level,
                "active": True
            })
            prev_level = level

        hierarchy_payload = {
            "tenantId": tenant,
            "hierarchyType": hierarchy_type,
            "boundaryHierarchy": boundary_items
        }

        try:
            result = uploader.create_boundary_hierarchy(hierarchy_payload)
            if result:
                print("\n➡️  Now generate the Excel template")
                generate_template_btn.layout.visibility = 'visible'
        except Exception as e:
            print(f"\n❌ Failed: {str(e)[:200]}")

create_hierarchy_btn.on_click(on_create_hierarchy_clicked)

def on_generate_template_clicked(b):
    global TEMPLATE_FILESTORE_ID

    with create_output:
        clear_output(wait=True)

        if not BOUNDARY_TENANT or not BOUNDARY_HIERARCHY_TYPE:
            print("❌ Please create hierarchy first")
            return

        print(f"📄 Generating template...")

        result = uploader.generate_boundary_template(BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE)
        if not result:
            print("❌ Generation failed")
            return

        print(f"\n⏳ Waiting for completion...")
        resource = uploader.poll_boundary_template_status(
            BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE, max_attempts=30, delay=2
        )

        if resource and resource.get('status') == 'completed':
            TEMPLATE_FILESTORE_ID = resource.get('fileStoreid')
            print(f"\n✅ Template ready!")
            download_template_btn.layout.visibility = 'visible'
        else:
            print("\n❌ Failed or timed out")

generate_template_btn.on_click(on_generate_template_clicked)

def on_download_template_clicked(b):
    global DOWNLOADED_TEMPLATE_PATH, DOWNLOAD_URL

    with create_output:
        if not TEMPLATE_FILESTORE_ID:
            print("❌ No template available")
            return

        print(f"\n📥 Downloading...")

        result = uploader.download_boundary_template(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=TEMPLATE_FILESTORE_ID,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            return_url=True
        )

        if result:
            DOWNLOADED_TEMPLATE_PATH = result['path']
            DOWNLOAD_URL = result['url']
            
            print(f"\n🎉 Downloaded!")
            print(f"\n" + "="*70)
            
            # Display clickable download link
            download_link_html = f"""
            <div style='background:#e7f3ff; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#0066cc;'>📥 Template Ready!</h3>
                <p><b>Local Path:</b> {DOWNLOADED_TEMPLATE_PATH}</p>
                <a href='{DOWNLOAD_URL}' download='boundary_template.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#007bff; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold; margin:10px 0;'>
                    ⬇️ Click Here to Download Template
                </a>
            </div>
            """
            display(HTML(download_link_html))
            
            print("📝 NEXT STEPS:")
            print("1. Click the download link above OR open file locally")
            print("2. Fill boundary data in Excel")
            print("3. Save the file")
            print("4. Proceed to Step 2.2 to upload")
            print("="*70)

download_template_btn.on_click(on_download_template_clicked)

# TAB 2 HANDLERS
def on_search_clicked(b):
    global BOUNDARY_TENANT

    with search_output:
        clear_output()

        tenant = tenant_input.value.strip()
        if not tenant:
            print("❌ Please enter tenant ID")
            return

        BOUNDARY_TENANT = tenant
        print(f"🔍 Searching hierarchies in: {tenant}")

        hierarchies = uploader.search_boundary_hierarchies(tenant)

        if hierarchies:
            hierarchy_options = [(f"{h['hierarchyType']} ({len(h.get('boundaryHierarchy', []))} levels)", h['hierarchyType']) for h in hierarchies]
            hierarchy_dropdown.options = hierarchy_options
            hierarchy_dropdown.layout.visibility = 'visible'
            search_generate_btn.layout.visibility = 'visible'
        else:
            print("\n⚠️  No hierarchies found")

search_btn.on_click(on_search_clicked)

def on_search_generate_clicked(b):
    global BOUNDARY_HIERARCHY_TYPE, TEMPLATE_FILESTORE_ID

    with search_output:
        clear_output(wait=True)

        hierarchy_type = hierarchy_dropdown.value
        if not hierarchy_type:
            print("❌ Please select hierarchy")
            return

        BOUNDARY_HIERARCHY_TYPE = hierarchy_type
        print(f"📄 Generating template for {hierarchy_type}...")

        result = uploader.generate_boundary_template(BOUNDARY_TENANT, hierarchy_type)
        if not result:
            return

        resource = uploader.poll_boundary_template_status(
            BOUNDARY_TENANT, hierarchy_type, max_attempts=30, delay=2
        )

        if resource and resource.get('status') == 'completed':
            TEMPLATE_FILESTORE_ID = resource.get('fileStoreid')
            print(f"\n✅ Ready!")
            search_download_btn.layout.visibility = 'visible'

search_generate_btn.on_click(on_search_generate_clicked)

def on_search_download_clicked(b):
    global DOWNLOADED_TEMPLATE_PATH, DOWNLOAD_URL

    with search_output:
        if not TEMPLATE_FILESTORE_ID:
            return

        result = uploader.download_boundary_template(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=TEMPLATE_FILESTORE_ID,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            return_url=True
        )

        if result:
            DOWNLOADED_TEMPLATE_PATH = result['path']
            DOWNLOAD_URL = result['url']
            
            # Display clickable download link
            download_link_html = f"""
            <div style='background:#e7f3ff; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#0066cc;'>📥 Template Ready!</h3>
                <a href='{DOWNLOAD_URL}' download='boundary_template.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#007bff; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold;'>
                    ⬇️ Click to Download Template
                </a>
            </div>
            """
            display(HTML(download_link_html))
            print(f"\n✅ Downloaded! Fill template and proceed to Step 2.2")

search_download_btn.on_click(on_search_download_clicked)

# Build Tab 1
add_initial_level()

tab1_content = widgets.VBox([
    widgets.HTML("<p style='color:#666;'><i>Create new hierarchy from scratch</i></p>"),
    hierarchy_type_input,
    widgets.HTML("<br><p style='font-weight:bold;'>🏛️ Boundary Levels:</p>"),
    levels_container,
    add_level_btn,
    widgets.HTML("<br>"),
    create_hierarchy_btn,
    generate_template_btn,
    download_template_btn,
    widgets.HTML("<br>"),
    create_output
])

# Build Tab 2
tab2_content = widgets.VBox([
    widgets.HTML("<p style='color:#666;'><i>Use existing hierarchy</i></p>"),
    search_btn,
    widgets.HTML("<br>"),
    hierarchy_dropdown,
    search_generate_btn,
    search_download_btn,
    widgets.HTML("<br>"),
    search_output
])

tab_selector.children = [tab1_content, tab2_content]
tab_selector.set_title(0, '🏗️ Create New')
tab_selector.set_title(1, '🔍 Use Existing')

main_ui = widgets.VBox([
    widgets.HTML("<h3>🔍 Step 2.1: Boundary Hierarchy & Template</h3>"),
    tenant_input,
    widgets.HTML("<br>"),
    tab_selector
])

display(main_ui)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from unified_loader import APIUploader
import os
import time

print("="*70)
print("   📤 PHASE 2.2: UPLOAD & PROCESS BOUNDARY DATA")
print("="*70)

UPLOADED_FILESTORE_ID = None
PROCESSED_FILESTORE_ID = None
PROCESSED_DOWNLOAD_URL = None


if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

uploader = UPLOADER

boundary_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

upload_process_btn = widgets.Button(
    description='📤 Upload & Process',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

check_status_btn = widgets.Button(
    description='🔍 Check Processing Status',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

download_processed_btn = widgets.Button(
    description='📥 Download Processed File',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

output = widgets.Output()

def on_upload_process_clicked(b):
    global UPLOADED_FILESTORE_ID, PROCESSED_FILESTORE_ID, PROCESSED_DOWNLOAD_URL

    with output:
        clear_output()

        if not BOUNDARY_TENANT:
            print("❌ Complete Step 2.1 first")
            return

        if not BOUNDARY_HIERARCHY_TYPE:
            print("❌ No hierarchy type selected")
            return

        if not boundary_file_upload.value:
            print("❌ Select filled template")
            return

        os.makedirs('upload', exist_ok=True)
        uploaded_file = boundary_file_upload.value[0]
        content = uploaded_file['content']
        filename = f'boundary_filled_{BOUNDARY_TENANT}_{BOUNDARY_HIERARCHY_TYPE}.xlsx'

        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)

        print(f"📄 Saved: {filename}")

        print(f"\n📤 Uploading to filestore...")
        filestore_id = uploader.upload_file_to_filestore(
            file_path=upload_path,
            tenant_id=BOUNDARY_TENANT,
            module='HCM-ADMIN-CONSOLE'
        )

        if not filestore_id:
            print("❌ Upload failed")
            return

        UPLOADED_FILESTORE_ID = filestore_id

        print(f"\n⚙️ Processing...")
        result = uploader.process_boundary_data(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=filestore_id,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            action='create'
        )

        if result:
            status = result.get('status')
            processed_id = result.get('processedFileStoreId')

            print(f"\n✅ Submitted")
            print(f"   Status: {status}")

            if status == 'data-accepted':
                print("\n" + "="*70)
                print("🎉 PHASE 2 COMPLETED!")
                print("="*70)
                print("✅ Boundaries created")
                print("\n⏳ Processing in background...")
                print("   Click 'Check Status' after 10-20 seconds")
                print("="*70)
                check_status_btn.layout.visibility = 'visible'
            elif processed_id:
                PROCESSED_FILESTORE_ID = processed_id
                download_processed_btn.layout.visibility = 'visible'
                print("\n💡 Processed file ready for download")

upload_process_btn.on_click(on_upload_process_clicked)

def on_check_status_clicked(b):
    global PROCESSED_FILESTORE_ID, PROCESSED_DOWNLOAD_URL
    
    with output:
        print("\n🔍 Checking processing status...")
        
        # Poll the _process endpoint to get status
        boundary_mgmt_url = uploader.boundary_mgmt_url
        url = f"{boundary_mgmt_url}/v1/_process-search"
        
        user_info_copy = uploader.user_info.copy()
        user_info_copy['tenantId'] = BOUNDARY_TENANT
        
        payload = {
            "RequestInfo": {
                "apiId": "Rainmaker",
                "authToken": uploader.auth_token,
                "userInfo": user_info_copy,
                "msgId": f"{int(time.time() * 1000)}|en_IN",
                "plainAccessRequest": {}
            },
            "SearchCriteria": {
                "tenantId": BOUNDARY_TENANT,
                "hierarchyType": BOUNDARY_HIERARCHY_TYPE
            }
        }
        
        try:
            import requests
            response = requests.post(url, json=payload, headers={'Content-Type': 'application/json'})
            response.raise_for_status()
            data = response.json()
            
            resources = data.get('ResourceDetails', [])
            if resources:
                latest = resources[0]
                status = latest.get('status')
                
                # Debug: show all available fields
                print(f"   Status: {status}")
                print(f"   Available fields: {list(latest.keys())}")
                
                # Try different field name variations
                processed_id = (latest.get('processedFileStoreId') or 
                               latest.get('processedFilestoreId') or 
                               latest.get('processedFilestoreid') or
                               latest.get('fileStoreId'))
                
                if processed_id:
                    PROCESSED_FILESTORE_ID = processed_id
                    print(f"   Processed File ID: {processed_id}")
                    
                    # Get download URL
                    filestore_url = uploader.filestore_url
                    file_url = f"{filestore_url}/v1/files/url"
                    params = {"tenantId": BOUNDARY_TENANT, "fileStoreIds": processed_id}
                    
                    file_response = requests.get(file_url, params=params)
                    file_response.raise_for_status()
                    file_data = file_response.json()
                    
                    file_urls = file_data.get('fileStoreIds', [])
                    if file_urls:
                        PROCESSED_DOWNLOAD_URL = file_urls[0].get('url')
                        
                        download_link_html = f"""
                        <div style='background:#d4edda; padding:15px; border-radius:8px; margin:10px 0;'>
                            <h3 style='margin-top:0; color:#155724;'>✅ Processed File Ready!</h3>
                            <p>The boundary data has been processed. Download the file to verify:</p>
                            <a href='{PROCESSED_DOWNLOAD_URL}' download='boundary_processed.xlsx' 
                               style='display:inline-block; padding:12px 24px; background:#28a745; color:white; 
                                      text-decoration:none; border-radius:6px; font-weight:bold;'>
                                📥 Download Processed File
                            </a>
                        </div>
                        """
                        display(HTML(download_link_html))
                        download_processed_btn.layout.visibility = 'visible'
                        check_status_btn.layout.visibility = 'hidden'
                elif status == 'inprogress':
                    print("\n⏳ Still processing... Check again in 10 seconds")
                else:
                    print(f"\n⚠️ Status: {status}")
            else:
                print("   No processing records found")
                
        except Exception as e:
            print(f"❌ Error: {str(e)[:200]}")

check_status_btn.on_click(on_check_status_clicked)

def on_download_processed_clicked(b):
    with output:
        if not PROCESSED_DOWNLOAD_URL:
            print("⚠️ No processed file URL available")
            return
        
        print(f"\n📥 Processed file URL ready!")
        print("   Click the green button above to download")

download_processed_btn.on_click(on_download_processed_clicked)

ui = widgets.VBox([
    widgets.HTML("<h3>📤 Step 2.2: Upload & Process</h3>"),
    widgets.HTML(f"<p style='background:#e7f3ff;padding:10px;border-radius:5px;'>"
                 f"<b>📋 Tenant:</b> {BOUNDARY_TENANT if BOUNDARY_TENANT else 'Not set'}<br>"
                 f"<b>🏛️ Hierarchy:</b> {BOUNDARY_HIERARCHY_TYPE if BOUNDARY_HIERARCHY_TYPE else 'Not set'}</p>"),
    widgets.HTML("<br><p><b>Select Filled Template:</b></p>"),
    boundary_file_upload,
    widgets.HTML("<p style='font-size:11px;color:#ff6600;'><i>⚠️ Make sure template is filled</i></p>"),
    widgets.HTML("<br>"),
    upload_process_btn,
    widgets.HTML("<br>"),
    check_status_btn,
    download_processed_btn,
    widgets.HTML("<br>"),
    output
])

display(ui)